In [1]:
import tensorflow as tf
tf.__version__

'2.17.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
import os
os.getcwd()

'/content'

In [8]:
import tensorflow as tf
from PIL import Image
import numpy as np
import cv2

In [6]:
target = ['Heart','Oblong','Oval','Round','Square']

In [9]:
test_images = '../content/drive/MyDrive/FaceShape Dataset/testing_set'
x_test = []
y_test = []
for (_,directories,_) in os.walk(test_images):
  for directory in directories:
    for(_,_,files) in os.walk(os.path.join(test_images,directory)):
      for filename in files:
        with open(os.path.join(test_images,directory,filename),'rb') as f:
          faceimage = cv2.imread(os.path.join(test_images,directory,filename))
          face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
          try:
            gray = cv2.cvtColor(faceimage,cv2.COLOR_BGR2GRAY)
          except:
            continue
          faces = face_cascade.detectMultiScale(gray)
          if len(faces)>0:
            x,y,w,h = faces[0]
          else:
            continue

          img = Image.open(f).convert('RGB').crop((x,y,x+w,y+h)).resize((380,380))
          arr = np.array(img)
          x_test.append(arr)
          y_test.append(directory)

In [10]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [11]:
print(x_test.shape)
print(y_test.shape)

(997, 380, 380, 3)
(997,)


In [12]:
mean_r, mean_g, mean_b= [159.80679614507162,122.67351405273274,104.61857584263046]
std_r, std_g, std_b= [72.58244780862275, 62.41943811258287, 59.047168710327774]

In [13]:
x_test[:,:,:,0] = (x_test[:,:,:,0] - mean_r)/float(std_r)
x_test[:,:,:,1] = (x_test[:,:,:,1] - mean_g)/float(std_g)
x_test[:,:,:,2] = (x_test[:,:,:,2] - mean_b)/float(std_b)

In [14]:
#target = ['Heart','Oblong','Oval','Round','Square']
# 값 변환을 위한 dictionary
mapping_dict = {'Heart': 0, 'Oblong': 1, 'Oval': 2, 'Round': 3,'Square':4}

# 벡터화된 함수를 사용하여 배열의 값을 변경
vectorized_map = np.vectorize(mapping_dict.get)
y_test = vectorized_map(y_test)

In [15]:
y_test = np.array(y_test)

In [16]:
interpreter = tf.lite.Interpreter(model_path='faceshape_efficientnetb4_crop.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [17]:
y_predict = []
for x in x_test:
    arr = np.expand_dims(x,axis=0)
    interpreter.set_tensor(input_details[0]['index'], arr.astype(np.float32))

    # 추론 실행
    interpreter.invoke()

    # 출력 텐서에서 결과 가져오기
    result = interpreter.get_tensor(output_details[0]['index'])
    #print("'Heart': 0, 'Oblong': 1, 'Oval': 2, 'Round': 3,'Square':4")
    #print(result[0])
    y_predict.append(result[0].argmax())

In [18]:
y_predict = np.array(y_predict)

In [19]:
np.sum(np.equal(y_test,y_predict)) / len(y_predict)

0.7853560682046138